In [13]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 8.5
sigmaH = 0.1
sigmaB = 0.01

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)
W_b = np.random.multivariate_normal(mean_p, sigmaB**2*identity_p)
print(W_b)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    # lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    mean_i = np.dot(W_d, feature_train[i, :])
    sigma_i = np.exp(np.dot(W_b, feature_train[i, :]))
    d_i = np.random.lognormal(mean_i, sigma_i)
    # d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
[ 0.00905602  0.00475985  0.00721022  0.00338268 -0.00280316  0.00334028
  0.00833226 -0.0121702 ]
mask proportion (within the original y_train = 1 )
20.016565433462176
prediction for base model
0.6076920935476612


In [14]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.025, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:35, xent:651.7670049667358
0.7049275263134696
[MF-IPS-DF] epoch:30, xent:652.6689786911011
0.7071064927050734
[MF-IPS-DF] epoch:26, xent:652.1572017669678
0.706957729095958
[MF-IPS-DF] epoch:30, xent:653.2065629959106
0.7042387092383845
[MF-IPS-DF] epoch:26, xent:652.9705400466919
0.7055703320318691
[MF-IPS-DF] epoch:30, xent:653.1038656234741
0.7087952834879547
[MF-IPS-DF] epoch:33, xent:653.7771310806274
0.7059642317766028
[MF-IPS-DF] epoch:32, xent:651.2478771209717
0.7083864977160581
[MF-IPS-DF] epoch:29, xent:653.6994724273682
0.7031807380221612
[MF-IPS-DF] epoch:31, xent:652.8343000411987
0.706539277072657
[MF_IPS_DF] test auc: 0.7061666817460187  sd:  0.0016748140480222068
[0.23192456 0.70616668 0.61109024 0.67666972 0.42987895 0.69952475]
[0.00032938 0.00167481 0.00695699 0.00442868 0.00890848 0.00591386]


In [15]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:24, xent:49762075.5625
0.7171914861204228
[MF-DR-JL-DF] epoch:25, xent:49881155.9375
0.7141442970276662
[MF-DR-JL-DF] epoch:21, xent:49796349.875
0.717637776947769
[MF-DR-JL-DF] epoch:19, xent:49917402.5
0.7269594264897047
[MF-DR-JL-DF] epoch:21, xent:49741937.875
0.7247303889158255
[MF-DR-JL-DF] epoch:20, xent:49812602.0
0.7445659234479287
[MF-DR-JL-DF] epoch:25, xent:49866575.5
0.7099417009045291
[MF-DR-JL-DF] epoch:26, xent:49846659.25
0.7282132359593833
[MF-DR-JL-DF] epoch:20, xent:49848758.125
0.7242955042651368
[MF-DR-JL-DF] epoch:22, xent:49812134.125
0.7390060889650847
[MF_DR_JL_DF] test auc: 0.724668582904345  sd:  0.010258677465356227
[0.33745513 0.72466858 0.61756337 0.6929773  0.43372315 0.72411167]
[0.00085242 0.01025868 0.0218948  0.01794607 0.01939594 0.01303427]


In [16]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-5,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:17, xent:57.05696576833725
0.6320995716690678
[MLP_weibull] epoch:15, xent:56.97569686174393
0.6318922305764411
[MLP_weibull] epoch:13, xent:56.977576434612274
0.6319568977983372
[MLP_weibull] epoch:17, xent:57.01525366306305
0.632065739789152
[MLP_weibull] epoch:22, xent:56.95767438411713
0.6319639541618626
[MLP_weibull] epoch:17, xent:56.97996234893799
0.6318424493817079
[MLP_weibull] epoch:18, xent:56.967029452323914
0.6321026648695172
[MLP_weibull] epoch:15, xent:57.000570356845856
0.6320321012342642
[MLP_weibull] epoch:14, xent:56.9975306391716
0.6319913096533373
[MLP_weibull] epoch:21, xent:56.993410646915436
0.6321226740099245
[MLP_weibull] test auc: 0.6320069593143611  sd:  8.946533648231673e-05
[0.2299329  0.63200696 0.52724817 0.60378234 0.36232001 0.64339875]
[1.14968074e-03 8.94653365e-05 1.53149073e-04 3.66054493e-04
 2.92991320e-04 5.32966580e-04]


In [22]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:13, xent:56556.689392089844
0.6738932528791896
[MF-DR] epoch:14, xent:56626.89373779297
0.6763601769001337
[MF-DR] epoch:16, xent:56613.96691894531
0.666817556851091
[MF-DR] epoch:16, xent:56585.284118652344
0.665985485930191
[MF-DR] epoch:12, xent:56545.10369873047
0.6855886437785299
[MF-DR] epoch:14, xent:56560.783142089844
0.689022869577523
[MF-DR] epoch:17, xent:56572.462646484375
0.6750039051655674
[MF-DR] epoch:16, xent:56631.259521484375
0.6823898878637507
[MF-DR] epoch:12, xent:56589.59423828125
0.6712917746386948
[MF-DR] epoch:13, xent:56568.56085205078
0.6978692295378992
[MF_DR] test auc: 0.6784222783122571  sd:  0.00966587928155964
[0.23456159 0.67842228 0.57949714 0.65560372 0.40087553 0.69048795]
[0.00036644 0.00966588 0.0127627  0.00967543 0.01496525 0.00815074]
